In [1]:
import pandas as pd

In [18]:
#Test
df = pd.DataFrame(data={'Item':['A','B','C','D'], 
                        'StartDate':['2018-01-01','2018-01-07', '2017-12-30','2018-01-03'],
                        'EndDate': ['2018-01-04','2018-01-08','2018-01-05','2018-01-05'],
                        'CT':[12,14,21,12]},
                 )
df['StartDate'] = pd.to_datetime(df['StartDate'])
df['EndDate'] = pd.to_datetime(df['EndDate'])
df['dayDiff'] = df.apply(lambda avg: avg['CT']/((avg['EndDate']-avg['StartDate']).days +1),axis=1)
print(df)


  Item  StartDate    EndDate  CT  dayDiff
0    A 2018-01-01 2018-01-04  12      3.0
1    A 2018-01-07 2018-01-08  14      7.0
2    B 2017-12-30 2018-01-05  21      3.0
3    A 2018-01-03 2018-01-05  12      4.0


In [47]:


index = pd.MultiIndex.from_product([['A','B'],pd.date_range(start = df.StartDate.min(), end = df.EndDate.max())], names=['item', 'date'])
new_df = pd.DataFrame(index = index, data = {'counts':0})

dates_list = new_df.index.get_level_values(1)
items_list = new_df.index.get_level_values(0)

for row in df.itertuples():
    new_df.counts[(items_list == row.Item) & (dates_list>=row.StartDate) & (dates_list<= row.EndDate)] += row[5]

# to remove 0 if needed
# new_df = new_df[(new_df.counts != 0)]

new_df.reset_index(inplace=True)
print(new_df)


   item       date  counts
0     A 2017-12-30       0
1     A 2017-12-31       0
2     A 2018-01-01       3
3     A 2018-01-02       3
4     A 2018-01-03       7
5     A 2018-01-04       7
6     A 2018-01-05       4
7     A 2018-01-06       0
8     A 2018-01-07       7
9     A 2018-01-08       7
10    B 2017-12-30       3
11    B 2017-12-31       3
12    B 2018-01-01       3
13    B 2018-01-02       3
14    B 2018-01-03       3
15    B 2018-01-04       3
16    B 2018-01-05       3
17    B 2018-01-06       0
18    B 2018-01-07       0
19    B 2018-01-08       0
